# Part 1 - Surgery Duration Prediction: Model Training

## 1. Setup & Data Loading

In [1]:
import pandas as pd
import numpy as np
import os, json, warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.compose import TransformedTargetRegressor
from sklearn.inspection import permutation_importance as sk_perm_imp
from sklearn.model_selection import train_test_split
from scipy.stats import randint, uniform
from xgboost import XGBRegressor
import joblib
warnings.filterwarnings('ignore')

os.makedirs('results', exist_ok=True)

df = pd.read_csv('surgeries to predict.csv')
print(f'Loaded: {df.shape[0]} rows x {df.shape[1]} columns')


Loaded: 10000 rows x 8 columns


## 2. Preprocessing

In [2]:
# Drop row index 
df_clean = df.drop(columns=['Unnamed: 0'])

# Keep raw Surgery Type for GroupMeanEncoder 
surgery_type_raw = df_clean['Surgery Type'].copy()

# One hot encode surgery type and anesthesia type 
surgery_dummies    = pd.get_dummies(df_clean['Surgery Type'],   prefix='surgery',   dtype=int)
anesthesia_dummies = pd.get_dummies(df_clean['Anesthesia Type'], prefix='anesthesia',
                                    drop_first=True, dtype=int)

df_clean = pd.concat([df_clean.drop(columns=['Surgery Type', 'Anesthesia Type']),
                       surgery_dummies, anesthesia_dummies], axis=1)
df_clean['surgery_type_raw'] = surgery_type_raw

# drop physiologically impossible BMI rows (< 10) before any split
mask_bad = df_clean['BMI'] < 10
n_removed = mask_bad.sum()
print(f'Rows with BMI < 10 removed: {n_removed} ({n_removed/len(df_clean)*100:.2f}%)')
df_clean = df_clean[~mask_bad].reset_index(drop=True)

print(f'Shape after cleaning: {df_clean.shape}')


Rows with BMI < 10 removed: 17 (0.17%)
Shape after cleaning: (9983, 12)


## 3. Feature Engineering

In [3]:
# Age and BMI interaction
df_clean['age_bmi'] = df_clean['Age'] * df_clean['BMI']

# Non linear transforms
df_clean['age_sq']    = df_clean['Age'] ** 2
df_clean['bmi_sq']    = df_clean['BMI'] ** 2
df_clean['log1p_bmi'] = np.log1p(df_clean['BMI'])

# Age and BMI buckets 
age_dummies = pd.get_dummies(
    pd.cut(df_clean['Age'], bins=[0, 30, 45, 60, np.inf],
           labels=['age_lt30', 'age_30_45', 'age_45_60', 'age_60plus']), dtype=int)
bmi_dummies = pd.get_dummies(
    pd.cut(df_clean['BMI'], bins=[0, 18.5, 25, 30, 35, np.inf],
           labels=['bmi_under18', 'bmi_18_25', 'bmi_25_30', 'bmi_30_35', 'bmi_35plus']), dtype=int)
df_clean = pd.concat([df_clean, age_dummies, bmi_dummies], axis=1)

# Surgery and patient interactions 
surgery_cols = [c for c in df_clean.columns
                    if c.startswith('surgery_') and c != 'surgery_type_raw']
for col in surgery_cols:
    df_clean[f'{col}_x_age'] = df_clean[col] * df_clean['Age']
    df_clean[f'{col}_x_bmi'] = df_clean[col] * df_clean['BMI']

print(f'Total features: {df_clean.shape[1] - 1}')


Total features: 34


## 4. Custom Transformers

In [4]:
class FrequencyEncoder(BaseEstimator, TransformerMixin):
    """Replaces each ID with its count in the training fold (no leakage)."""
    def __init__(self, columns): self.columns = columns
    def fit(self, X, y=None):
        self.frequency_maps_ = {col: X[col].value_counts().to_dict() for col in self.columns}
        return self
    def transform(self, X):
        X = X.copy()
        for col in self.columns:
            X[col] = X[col].map(self.frequency_maps_[col]).fillna(0).astype(int)
        return X


class GroupMeanEncoder(BaseEstimator, TransformerMixin):
    """Fold safe target encoding: adds mean per group value, drops raw column."""
    def __init__(self, columns): self.columns = columns
    def fit(self, X, y):
        y_arr = np.asarray(y, dtype=float)
        self.global_mean_ = float(y_arr.mean())
        self.mean_maps_ = {}
        for col in self.columns:
            tmp = pd.DataFrame({'g': np.asarray(X[col]), 'y': y_arr})
            self.mean_maps_[col] = tmp.groupby('g')['y'].mean().to_dict()
        return self
    def transform(self, X):
        X = X.copy()
        for col in self.columns:
            X[f'{col}_mean_duration'] = X[col].map(self.mean_maps_[col]).fillna(self.global_mean_)
            X = X.drop(columns=[col])
        return X



## 5. Model Definitions

In [5]:
ID_COLS       = ['DoctorID', 'AnaesthetistID']
MEAN_ENC_COLS = ['surgery_type_raw']

X = df_clean.drop(columns=['Duration in Minutes'])
y = df_clean['Duration in Minutes']
groups = X['DoctorID']   # GroupKFold:the same DoctorID never appears in both train and validation

def make_pipeline(model, scale=False):
    steps = [('freq_enc', FrequencyEncoder(columns=ID_COLS)),
             ('mean_enc', GroupMeanEncoder(columns=MEAN_ENC_COLS))]
    if scale: steps.append(('scaler', StandardScaler()))
    steps.append(('model', model))
    return Pipeline(steps)

models = {
    'Random Forest':     make_pipeline(RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)),
    'Gradient Boosting': make_pipeline(HistGradientBoostingRegressor(max_iter=200, random_state=42)),
    'XGBoost':           make_pipeline(XGBRegressor(n_estimators=200, random_state=42, n_jobs=-1, verbosity=0)),
    'Ridge Regression':  make_pipeline(Ridge(alpha=1.0), scale=True),
}


## 6. Default 5-Fold GroupKFold CV

In [6]:
kf = GroupKFold(n_splits=5)
results = {}

for model_name, pipeline in models.items():
    fold_mae, fold_mape, fold_acc15 = [], [], []
    for train_idx, val_idx in kf.split(X, y, groups):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        pipeline.fit(X_tr, y_tr)
        preds = pipeline.predict(X_val)
        fold_mae.append(np.mean(np.abs(preds - y_val)))
        fold_mape.append(np.mean(np.abs((preds - y_val) / y_val)) * 100)
        fold_acc15.append(np.mean(np.abs(preds - y_val) <= 15) * 100)
    results[model_name] = {
        'MAE (min)':      round(np.mean(fold_mae),   2),
        'MAE std':        round(np.std(fold_mae),    2),
        'MAPE (%)':       round(np.mean(fold_mape),  2),
        'Acc ±15min (%)': round(np.mean(fold_acc15), 2),
    }
    print(f'{model_name}: MAE={results[model_name]["MAE (min)"]}, Acc±15={results[model_name]["Acc ±15min (%)"]}%')

results_df = pd.DataFrame(results).T
results_df


Random Forest: MAE=17.39, Acc±15=49.25%
Gradient Boosting: MAE=16.14, Acc±15=49.66%
XGBoost: MAE=16.91, Acc±15=49.31%
Ridge Regression: MAE=15.94, Acc±15=49.54%


,MAE (min),MAE std,MAPE (%),Acc ±15min (%)
Random Forest,17.39,0.26,20.68,49.25
Gradient Boosting,16.14,0.30,19.30,49.66
XGBoost,16.91,0.29,20.14,49.31
Ridge Regression,15.94,0.20,19.81,49.54


## 7. RandomizedSearchCV 

In [7]:
param_grids = {
    'Random Forest': {
        'model__n_estimators':      randint(100, 500),
        'model__max_depth':         [None, 10, 20, 30],
        'model__min_samples_split': randint(2, 20),
        'model__min_samples_leaf':  randint(1, 10),
        'model__max_features':      ['sqrt', 'log2', 0.5],
    },
    'Gradient Boosting': {
        'model__max_iter':           randint(100, 400),
        'model__learning_rate':      uniform(0.01, 0.29),
        'model__max_depth':          [3, 5, 7, None],
        'model__min_samples_leaf':   randint(10, 50),
        'model__l2_regularization':  uniform(0.0, 2.0),
    },
    'XGBoost': {
        'model__n_estimators':    randint(100, 500),
        'model__learning_rate':   uniform(0.01, 0.29),
        'model__max_depth':       randint(3, 10),
        'model__subsample':       uniform(0.6, 0.4),
        'model__colsample_bytree': uniform(0.6, 0.4),
        'model__reg_alpha':       uniform(0.0, 2.0),
        'model__reg_lambda':      uniform(0.5, 2.5),
    },
    'Ridge Regression': {'model__alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]},
}

search_cv = GroupKFold(n_splits=3)
best_params_found = {}
for model_name, pipeline in models.items():
    search = RandomizedSearchCV(pipeline, param_grids[model_name], n_iter=20,
                                scoring='neg_mean_absolute_error', cv=search_cv,
                                random_state=42, n_jobs=-1, refit=False)
    search.fit(X, y, groups=groups)
    best_params_found[model_name] = search.best_params_
    print(f'{model_name}: best CV MAE={-search.best_score_:.2f}')


Random Forest: best CV MAE=15.90
Gradient Boosting: best CV MAE=15.88
XGBoost: best CV MAE=15.80
Ridge Regression: best CV MAE=15.95


## 8. Tuned Models (5-fold CV)

In [8]:
eval_cv = GroupKFold(n_splits=5)
tuned_results  = {}
tuned_pipelines = {}

for model_name, pipeline in models.items():
    tuned_pipeline = clone(pipeline)
    tuned_pipeline.set_params(**best_params_found[model_name])
    tuned_pipelines[model_name] = tuned_pipeline
    fold_mae, fold_mape, fold_acc15 = [], [], []
    for train_idx, val_idx in eval_cv.split(X, y, groups):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        tuned_pipeline.fit(X_tr, y_tr)
        preds = tuned_pipeline.predict(X_val)
        fold_mae.append(np.mean(np.abs(preds - y_val)))
        fold_mape.append(np.mean(np.abs((preds - y_val) / y_val)) * 100)
        fold_acc15.append(np.mean(np.abs(preds - y_val) <= 15) * 100)
    tuned_results[model_name] = {
        'MAE (min)':      round(np.mean(fold_mae),   2),
        'MAE std':        round(np.std(fold_mae),    2),
        'MAPE (%)':       round(np.mean(fold_mape),  2),
        'Acc ±15min (%)': round(np.mean(fold_acc15), 2),
    }

tuned_df = pd.DataFrame(tuned_results).T
print(tuned_df[['MAE (min)', 'MAPE (%)', 'Acc ±15min (%)']].to_string())


                   MAE (min)  MAPE (%)  Acc ±15min (%)
Random Forest          15.85     18.96           49.48
Gradient Boosting      15.78     19.00           49.75
XGBoost                15.74     18.92           49.99
Ridge Regression       15.94     19.81           49.54


## 9. Log Target Transform

In [9]:
log_results = {}
for model_name, tuned_pipeline in tuned_pipelines.items():
    log_model = TransformedTargetRegressor(regressor=clone(tuned_pipeline),
                                           func=np.log1p, inverse_func=np.expm1)
    fold_mae, fold_mape, fold_acc15 = [], [], []
    for train_idx, val_idx in eval_cv.split(X, y, groups):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        log_model.fit(X_tr, y_tr)
        preds = log_model.predict(X_val)
        fold_mae.append(np.mean(np.abs(preds - y_val)))
        fold_mape.append(np.mean(np.abs((preds - y_val) / y_val)) * 100)
        fold_acc15.append(np.mean(np.abs(preds - y_val) <= 15) * 100)
    log_results[model_name] = {
        'MAE (min)':      round(np.mean(fold_mae),   2),
        'MAPE (%)':       round(np.mean(fold_mape),  2),
        'Acc ±15min (%)': round(np.mean(fold_acc15), 2),
    }

log_df = pd.DataFrame(log_results).T
print(log_df[['MAE (min)', 'MAPE (%)', 'Acc ±15min (%)']].to_string())


                   MAE (min)  MAPE (%)  Acc ±15min (%)
Random Forest          15.92     18.41           49.57
Gradient Boosting      15.87     18.43           49.66
XGBoost                15.82     18.34           49.86
Ridge Regression       16.04     19.05           49.92


In [ ]:

# Compare all evaluations
print("=" * 100)
print("MODEL COMPARISON ACROSS ALL EVALUATIONS")
print("=" * 100)

comparison_df = pd.DataFrame({
    'Tuned (Step 8)': tuned_df['Acc ±15min (%)'],
    'Log Transform (Step 9)': log_df['Acc ±15min (%)'],
    'Improvement': log_df['Acc ±15min (%)'] - tuned_df['Acc ±15min (%)'],
})
comparison_df = comparison_df.sort_values('Log Transform (Step 9)', ascending=False)
print("\nAccuracy Comparison (±15 min) - Higher is Better:")
print(comparison_df)


MODEL COMPARISON ACROSS ALL EVALUATIONS

Accuracy Comparison (±15 min) - Higher is Better:
                   Tuned (Step 8)  Log Transform (Step 9)  Improvement
Ridge Regression            49.54                   49.92         0.38
XGBoost                     49.99                   49.86        -0.13
Gradient Boosting           49.75                   49.66        -0.09
Random Forest               49.48                   49.57         0.09


## 10. Feature Importance (best model)

In [12]:
# Select best model (highest Acc ±15min across tuned and log-transform runs)
best_acc_tuned  = tuned_df['Acc ±15min (%)'].max()
best_acc_log    = log_df['Acc ±15min (%)'].max()
use_log         = best_acc_log >= best_acc_tuned
best_model_name = (log_df if use_log else tuned_df)['Acc ±15min (%)'].idxmax()
best_acc        = max(best_acc_tuned, best_acc_log)
config_label    = f'{best_model_name} + log(y)' if use_log else best_model_name
print(f'Best model: {config_label}  |  Acc ±15min: {best_acc:.2f}%')


Best model: XGBoost  |  Acc ±15min: 49.99%


In [13]:

# Fit best pipeline on all data (without log transform)
best_pipeline_final = clone(tuned_pipelines[best_model_name])
best_pipeline_final.fit(X, y)

# Also fit log-transform version
final_model = TransformedTargetRegressor(
    regressor=clone(best_pipeline_final),
    func=np.log1p, inverse_func=np.expm1
)
final_model.fit(X, y)

# Extract feature importances
model_step = best_pipeline_final.named_steps['model']
X_preprocessed = best_pipeline_final[:-1].transform(X)
feature_names = list(X_preprocessed.columns) if hasattr(X_preprocessed, 'columns') \
                else [f'f{i}' for i in range(X_preprocessed.shape[1])]

if hasattr(model_step, 'feature_importances_'):
    method = 'built-in'
    feat_imp = pd.Series(model_step.feature_importances_, index=feature_names).sort_values(ascending=False)
else:
    method = 'permutation'
    result = sk_perm_imp(best_pipeline_final, X, y, n_repeats=10, random_state=42, n_jobs=-1)
    feat_imp = pd.Series(result.importances_mean, index=feature_names).sort_values(ascending=False)

print(f"Feature importance method: {method}")
print(f"\nTop 5 Most Important Features:")
print(feat_imp.head(5).to_string())


Feature importance method: built-in

Top 5 Most Important Features:
surgery_2                         0.445338
surgery_type_raw_mean_duration    0.277390
surgery_0                         0.216386
surgery_0_x_age                   0.018986
age_60plus                        0.012683


## 11. Save Results to `results/`

In [14]:
# DataFrames
results_df.to_csv('results/default_results.csv')
tuned_df.to_csv('results/tuned_results.csv')
log_df.to_csv('results/log_results.csv')
feat_imp.to_frame('importance').to_csv('results/feature_importances.csv')

def to_python(v):
    if isinstance(v, np.bool_):    return bool(v)
    if isinstance(v, np.integer):  return int(v)
    if isinstance(v, np.floating): return float(v)
    return v

# Best model metadata
meta = {
    'best_model_name': str(best_model_name),
    'config_label':    str(config_label),
    'use_log':         bool(use_log),
    'best_acc':        float(best_acc),
    'method':          str(method),
    'best_params':     {k: to_python(v) for k, v in best_params_found[best_model_name].items()},
    'top5':            {str(k): float(v) for k, v in feat_imp.head(5).items()},
}
with open('results/best_model_info.json', 'w') as f:
    json.dump(meta, f, indent=2)

# Fitted best model 
save_model = final_model if use_log else best_pipeline_final
joblib.dump(save_model, 'results/best_model.joblib')

print('Saved to results/:')
for fname in sorted(os.listdir('results')):
    size = os.path.getsize(f'results/{fname}')
    print(f'  {fname}  ({size:,} bytes)')


Saved to results/:
  best_model.joblib  (673,215 bytes)
  best_model_info.json  (696 bytes)
  default_results.csv  (195 bytes)
  feature_importances.csv  (857 bytes)
  log_results.csv  (170 bytes)
  tuned_results.csv  (196 bytes)
